In [1]:
%run ../prepare_data.py -N_users 1

D:\Users\greg_\Anaconda3\lib\site-packages\pandas\core\indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
D:\Users\greg_\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [2]:
sequence = transform_users(
    users.user_id.sample(1000)
)
df_train = sequence

print(df_train.shape)

D:\Users\greg_\Anaconda3\lib\site-packages\pandas\core\indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
D:\Users\greg_\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(187426, 82)


In [3]:
sequence = transform_users(
    users.user_id.sample(200)
)
df_test = sequence

print(df_test.shape)

D:\Users\greg_\Anaconda3\lib\site-packages\pandas\core\indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
D:\Users\greg_\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(33408, 82)


In [5]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.optimizers import RMSprop, adam
from keras.utils.data_utils import get_file
from keras.callbacks import History, ModelCheckpoint
import numpy as np
import pandas as pd
import random
import sys
import string
import matplotlib.pyplot as plt
%matplotlib inline


# Define number of inputs/outputs to handle
input_size = 16
output_size= 16

# Load the train data
...

print('Train total outputs:', output_size)

# Pre-process the data (normalize)

# cut the input in semi-redundant sequences of maxlen characters
maxlen = 90
step_days = 1



def reformat(df_in):
    max_sequences = 2*10**5
    days_sequence = np.empty((max_sequences,maxlen,input_size),dtype=int)
    next_day = np.empty((max_sequences,output_size),dtype=int)
    days = df_in[:,-1]
    df = df_in[:,:-1]
    
    j=0
    last_day = 0
    for day_i in range(0, df.shape[0] - maxlen, step_days):
        if last_day < days[day_i+maxlen]:
            days_sequence[j] = df[day_i: day_i + maxlen,:input_size]
        next_day[j] = df[day_i + maxlen,:output_size]
        j += 1
        last_day = days[day_i]

    days_sequence = days_sequence[:j,:,:]
    next_day = next_day[:j,:]
    
    return days_sequence,next_day

X_train, y_train = reformat(df_train)
X_test,y_test=reformat(df_test)

print(df_train.shape,df_test.shape)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

Train total outputs: 16
(187426, 82) (33408, 82)
(187336, 90, 16)
(187336, 16)
(33318, 90, 16)
(33318, 16)


In [6]:
def plot_logs(history):
    """
    Plot the accuracy and loss for 
        training and test sets
    """
    evaluation_cost = history.history['val_loss']
    evaluation_accuracy = history.history['val_acc']
    training_cost = history.history['loss']
    training_accuracy = history.history['acc']
    f, (ax1, ax2) = plt.subplots(1, 2)
    f.set_figwidth(10)
    ax1.plot(evaluation_cost,label= 'test')
    ax1.plot(training_cost, label='train')
    ax1.set_title('Cost')
    ax1.legend()
    ax2.plot(evaluation_accuracy, label='test')
    ax2.plot(training_accuracy, label='train')
    ax2.set_title('Accuracy')
    ax2.legend(loc='lower right')


def sample(preds, temperature=1.0):
    """
    Generate the next sequence
    Low temperature means very conservative (picks more probable most of the time)
    High temperature means very adventurous (picks less probable more frequently)
    """
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_prediction(history,days=28, maxlen=60, input_size=16, output_size=16, diversity=1):
    """
    Generates as many days of prediction as requested
    Considers maxlen days of past history (must be aligned with model)
    """
    generated = np.zeros((days,output_size))
    if history.shape[1]>maxlen:
        x = history[:,-maxlen-1:-1,:input_size]
    else:
        x = history[:,:,:input_size]
    #print(x.shape)
    for i in range(days):
        #print("Day %d" % i)
        preds = model.predict(x, verbose=0)[0].reshape(output_size)
        #print(preds)
        generated[i,:] = preds
        
        if input_size > output_size:
            res = np.zeros(input_size)
            res[:output_size] = preds
            preds = res

        #print(preds.shape)
        #next_symptoms = sample(preds, diversity)
        next_symptoms = preds
        #print(next_symptoms)


        x[:,:maxlen-1,:] = x[:,1:,:]
        x[:,maxlen-1,:] = next_symptoms

    return generated

In [8]:
# Model params
batch_size = 512
nb_epoch = 50
np.random.seed(131078)

# build the model
print('Build model...')


filepath="lstm_1_layer.hdf5"
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, input_size)))
model.add(Dense(output_size))
model.add(Activation('sigmoid'))

"""
filepath="lstm_2_layers_higher_dropout.hdf5"
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, input_size), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(256))
model.add(Dropout(0.6))
model.add(Dense(output_size))
model.add(Activation('sigmoid'))
"""

optimizer = adam()
model.compile(loss='binary_crossentropy', 
              optimizer=optimizer,
              metrics=['accuracy'])

# Define callback to save model
save_snapshots = ModelCheckpoint(filepath,
                                 monitor='loss',
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='min',
                                 verbose=0)
callbacks_list = [save_snapshots]

history = model.fit(X_train,
                    y_train,
                    batch_size=batch_size,
                    nb_epoch=nb_epoch,
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=1)          

score = model.evaluate(X_test, y_test, verbose=2)
print('Test score:', score[0])
print('Test accuracy:', score[1])  

plot_logs(history)          


Build model...
Train on 187336 samples, validate on 33318 samples
Epoch 1/50
187336/187336 [==============================] - 82s - loss: 0.0954 - acc: 0.9804 - val_loss: 0.0731 - val_acc: 0.9851
Epoch 2/50
187336/187336 [==============================] - 81s - loss: 0.0738 - acc: 0.9846 - val_loss: 0.0702 - val_acc: 0.9851
Epoch 3/50
187336/187336 [==============================] - 81s - loss: 0.0717 - acc: 0.9846 - val_loss: 0.0694 - val_acc: 0.9851
Epoch 4/50
187336/187336 [==============================] - 81s - loss: 0.0710 - acc: 0.9846 - val_loss: 0.0689 - val_acc: 0.9851
Epoch 5/50
187336/187336 [==============================] - 81s - loss: 0.0701 - acc: 0.9846 - val_loss: 0.0677 - val_acc: 0.9851
Epoch 6/50
187336/187336 [==============================] - 81s - loss: 0.0694 - acc: 0.9846 - val_loss: 0.0684 - val_acc: 0.9851
Epoch 7/50
187336/187336 [==============================] - 81s - loss: 0.0690 - acc: 0.9846 - val_loss: 0.0675 - val_acc: 0.9851
Epoch 8/50
187336/187336

KeyboardInterrupt: 

In [49]:
hist = X_test[0,:,:].reshape(1,maxlen,-1)
print(hist.shape)
res = generate_prediction(hist,maxlen=maxlen,input_size=input_size,output_size=output_size)
res.shape

(1, 10, 81)


(28, 16)

In [50]:
res[:,:output_size]

array([[ 0.01095976,  0.00664205,  0.00363475,  0.01084435,  0.00173918,
         0.00605508,  0.00538483,  0.01100782,  0.0215272 ,  0.00720003,
         0.00308284,  0.00900103,  0.00516408,  0.00340249,  0.00277895,
         0.00186608],
       [ 0.01095976,  0.00664205,  0.00363475,  0.01084435,  0.00173918,
         0.00605508,  0.00538483,  0.01100782,  0.0215272 ,  0.00720003,
         0.00308284,  0.00900103,  0.00516408,  0.00340249,  0.00277895,
         0.00186608],
       [ 0.01095976,  0.00664205,  0.00363475,  0.01084435,  0.00173918,
         0.00605508,  0.00538483,  0.01100782,  0.0215272 ,  0.00720003,
         0.00308284,  0.00900103,  0.00516408,  0.00340249,  0.00277895,
         0.00186608],
       [ 0.01095976,  0.00664205,  0.00363475,  0.01084435,  0.00173918,
         0.00605508,  0.00538483,  0.01100782,  0.0215272 ,  0.00720003,
         0.00308284,  0.00900103,  0.00516408,  0.00340249,  0.00277895,
         0.00186608],
       [ 0.01095976,  0.00664205,  0

In [51]:
y_test[:28,:output_size]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0,

In [137]:
def format_prediction(prediction,user_id):
    output = []
    prediction = pd.DataFrame(prediction)
    for i,row in prediction.iterrows():
        for j, symptom in enumerate(row):
            line = [user_id,i+1,j,prediction.ix[i,j]]
            output.append(line)
    return output

def pad_reshape_history(sequence,maxlen,input_size):
    if sequence.shape[0] < maxlen:
        hist = np.zeros((maxlen,sequence.shape[1]))
        hist[maxlen-sequence.shape[0]:,:] = sequence
    else:
        hist = sequence[-maxlen-1:-1,:]
    if sequence.shape[1] > input_size:
        hist = hist[:,:input_size]
    hist = hist.reshape(1,maxlen,-1)
    return hist
    
symptoms_of_interest_dict = {
    0:'happy',
    1:'pms',
    2:'sad', 
    3:'sensitive_emotion',
    4:'energized', 
    5:'exhausted', 
    6:'high_energy', 
    7:'low_energy',
    8:'cramps', 
    9:'headache', 
    10:'ovulation_pain', 
    11:'tender_breasts',
    12:'acne_skin', 
    13:'good_skin', 
    14:'oily_skin', 
    15:'dry_skin'
}
        
submission = []
j = 0
for index, woman in cycles0.iterrows():
    current_id = woman.user_id
    expected_length = int(np.ceil(woman.expected_cycle_length))
    sequence = transform_user(current_id)
    hist = pad_reshape_history(sequence,maxlen,input_size)
    res = generate_prediction(hist,maxlen=maxlen,input_size=input_size,output_size=output_size,days=expected_length)
    submission.append(format_prediction(res,current_id))
    j+=1
    if j > 2:
        break   

D:\Users\greg_\Anaconda3\lib\site-packages\pandas\core\indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
D:\Users\greg_\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [138]:
submission_df = pd.concat([pd.DataFrame(submission[i]) for i in range(len(submission))], ignore_index=True)  
submission_df.columns = ['user_id','day_in_cycle','symptom','probability']
submission_df["symptom"] = submission_df["symptom"].apply(lambda x: symptoms_of_interest_dict[x])    

submission_df.to_csv("results.csv",index=False) 

In [139]:
submission_df

,user_id,day_in_cycle,symptom,probability
0,03009100-a1fa-4fad-bf9b-6102c690f3be,1,happy,0.013489
1,03009100-a1fa-4fad-bf9b-6102c690f3be,1,pms,0.009573
2,03009100-a1fa-4fad-bf9b-6102c690f3be,1,sad,0.006829
3,03009100-a1fa-4fad-bf9b-6102c690f3be,1,sensitive_emotion,0.016105
4,03009100-a1fa-4fad-bf9b-6102c690f3be,1,energized,0.002734
5,03009100-a1fa-4fad-bf9b-6102c690f3be,1,exhausted,0.004730
6,03009100-a1fa-4fad-bf9b-6102c690f3be,1,high_energy,0.008621
7,03009100-a1fa-4fad-bf9b-6102c690f3be,1,low_energy,0.015173
8,03009100-a1fa-4fad-bf9b-6102c690f3be,1,cramps,0.027564
9,03009100-a1fa-4fad-bf9b-6102c690f3be,1,headache,0.012830
